# 🚀 Projeto MVP: Análise de Rotatividade de Funcionários (HR Attrition) - Spring Engenharia de Dados

**Nome:** Vitor Teixeira Moraes

**Matrícula:** 4052025000694

**--- OBSERVAÇÕES IMPORTANTES ---**

**Distinção entre os Sprints**

Para contextualizar este trabalho, é importante diferenciar os objetivos de cada sprint, principalmente as que já foram realizadas anteriormente:

- **MVP Primeira Sprint** (Análise de Dados & Boas Práticas): O foco foi realizar uma Análise Exploratória de Dados (EDA) completa sobre o dataset de attrition da IBM. Naquele projeto, investigamos a fundo a estrutura dos dados, as correlações entre as variáveis, o perfil dos funcionários e geramos visualizações para extrair os primeiros insights sobre os possíveis fatores que influenciam a rotatividade. O trabalho anterior pode ser acessado neste [repositório do GitHub.](https://github.com/vitortmoraes/vitormoraes_mvp_2025-1)

- **MVP Segunda Sprint** (Machine Learning & Analytics): Partindo dos insights gerados pela análise exploratória, o objetivo foi de construir, treinar e avaliar modelos preditivos. Aplicamos técnicas de Machine Learning para criar um sistema capaz de prever quais funcionários têm maior probabilidade de deixar a empresa, focando na implementação de um fluxo de trabalho robusto e na comparação de diferentes algoritmos para encontrar a solução mais eficaz. [repositório do GitHub.](https://github.com/vitortmoraes/vitormoraes_mvp_2025-2)

### 1. Objetivo e Definição do Problema

Este MVP foi desenvolvido para fornecer inteligência de negócio (BI) à área de Recursos Humanos (RH), focando no problema da **Rotatividade de Funcionários (Attrition)**. O trabalho representa uma evolução dos projetos anteriores, reutilizando o contexto de negócio.

**Links de Projetos Anteriores (Contexto da Pós-Graduação):**
* Projeto MVP 2025-1: [https://github.com/vitortmoraes/vitortmoraes_mvp_2025-1](https://github.com/vitortmoraes/vitormoraes_mvp_2025-1.git)
* Projeto MVP 2025-2: [https://github.com/vitortmoraes/vitortmoraes_mvp_2025-2](https://github.com/vitortmoraes/vitormoraes_mvp_2025-2.git)

O objetivo é realizar análises de diagnóstico para identificar as variáveis (demográficas, de cargo e financeiras) que mais influenciam a decisão do funcionário de deixar a empresa.

**Justificativa de MVP:**
Este projeto migra da análise pontual (OLTP) para uma **arquitetura analítica (OLAP)**. O MVP é a **versão mínima viável** para criar uma **base histórica e integrada** de dados em um ambiente de Big Data (Spark/Databricks), suportando as decisões gerenciais.

**Perguntas de Negócio a serem Respondidas (Análise OLAP):**
1. Qual departamento possui a maior taxa de rotatividade (*Attrition Rate*)? (Foco: Vendas vs. P&D)
2. Qual é o perfil comportamental (Satisfação e Envolvimento) predominante entre os funcionários que pediram demissão?
3. Qual é a renda mensal média dos funcionários que saem versus a dos que permanecem?

---

## 2. Busca e Coleta de Dados (Data Sourcing)

O conjunto de dados utilizado, "IBM HR Analytics Employee Attrition & Performance" (`WA_Fn-UseC_HR_Employee_Attrition.csv`), é proveniente do repositório Kaggle (Dados Abertos) que foi inserido no Pipeline do Databricks.

### Tecnologia e Conformidade (GGD)

* **Plataforma Utilizada:** Databricks Community Edition (Ambiente Cloud-Native/Hadoop-Spark).
* **Conceito de Governança (GGD):** O Databricks atua como o ambiente de **Metadados** central, onde a tabela é gerenciada (Catálogo Unity/Metastore). Essa centralização das definições do dado (como tipo, domínio e acesso) é fundamental para a **Gestão e Governança de Dados** (DMBOK).
* **Conformidade:** O *dataset* é considerado de **Dados Abertos**, e o processo de coleta e ingestão demonstra o uso ético dos dados, respeitando as licenças de uso.

---

## 3. Modelagem de Dados: Esquema Estrela

O conjunto de dados original é um *flat file* (tabela única). Para otimizar as consultas analíticas (OLAP) e garantir a eficiência no Databricks, a modelagem lógica escolhida é o **Esquema Estrela** (Star Schema), que é ideal para Data Warehouses.

A Modelagem Dimensional envolve a desnormalização do *flat file* em uma Tabela Fato central e múltiplas Dimensões.

### 3.1. Catálogo de Dados e Domínios (DMBOK)

Conforme a exigência de Modelagem, o Catálogo de Dados (Metadados) define a estrutura e o domínio de valores. A **Linhagem do Dado** aqui é o próprio processo ETL que mapeia as colunas de origem para o Esquema Estrela.

| Atributo (Coluna) | Tabela (Destino) | Tipo (Spark) | Domínio/Regra de Qualidade (Semântica) | Função de Chave |
| :--- | :--- | :--- | :--- | :--- |
| **Employee\_SK** | DIM\_EMPLOYEE, FACT\_HR\_MOVEMENT | BIGINT | Numérico e Sequencial. | **SK / PK (DIM) / FK (FACT)** |
| **Job\_SK** | DIM\_JOB\_INFO, FACT\_HR\_MOVEMENT | BIGINT | Numérico e Sequencial. | **SK / PK (DIM) / FK (FACT)** |
| **Attrition\_Flag** | FACT\_HR\_MOVEMENT | INT | Apenas **1** (Saiu) ou **0** (Ficou). | Métrica de Fato |
| **Age** | DIM\_EMPLOYEE | BIGINT | Faixa de Validade: **18 <= Idade <= 60**. | Atributo de Dimensão |
| **Department** | DIM\_JOB\_INFO | STRING | Categórico. Domínio: Sales, R&D, HR. | Atributo de Dimensão |
| **MonthlyIncome** | DIM\_JOB\_INFO | DOUBLE | Validade: Valores **positivos** e **não nulos**. | Atributo de Dimensão |
| **EmployeeNumber** | DIM\_EMPLOYEE, DIM\_JOB\_INFO | BIGINT | Chave de Negócio. Não nulo. | Chave de Negócio |

> **Nota de Linhagem:** As Chaves Substitutas (SKs) e o `Attrition_Flag` são **atributos derivados** criados na etapa de Transformação (ETL) para otimizar o processamento analítico.

In [0]:
# 1. Definindo o nome completo da sua tabela persistida
TABELA_ORIGINAL = "`workspace`.`mvp-2025-3`.`wa_fn_use_c_hr_employee_attrition`"

# 2. Criando uma view temporária diretamente da tabela
# Isso permite que os comandos SQL subsequentes acessem a tabela facilmente.
spark.sql(f"CREATE OR REPLACE TEMPORARY VIEW raw_attrition AS SELECT * FROM {TABELA_ORIGINAL}")

print(f"View temporária 'raw_attrition' criada a partir da tabela: {TABELA_ORIGINAL}")

## 4. Carga de Dados e Pipeline ETL (Extração, Transformação e Carga)

A fase de ETL transforma o dado bruto (`raw_attrition`) no **Esquema Estrela**. As transformações incluem a desnormalização e a criação de **Chaves Substitutas (SKs)**, ligando as Dimensões à Fato, e o **Pré-processamento** da métrica principal.

**Conceito de Linhagem:** O código abaixo atua como a **Linhagem do Dado**, documentando o fluxo de transformação do *flat file* (Origem) para as tabelas analíticas (Destino) no formato Delta Lake.

### 4.1. CRIAÇÃO DA DIMENSÃO DO FUNCIONÁRIO (DIM_EMPLOYEE)

Criação de Chave Substituta (SK) e seleção dos atributos de perfil.

In [0]:
%sql
-- Cria a tabela DIM_EMPLOYEE no Catálogo Databricks
CREATE OR REPLACE TABLE DIM_EMPLOYEE USING DELTA AS
SELECT
    -- Gerando a Chave Substituta (SK) para a Dimensão, o Databricks/Spark garante a unicidade.
    row_number() OVER (ORDER BY EmployeeNumber) AS Employee_SK,
    
    -- Chave de Negócio: identificador que liga a esta e outras dimensões
    EmployeeNumber,
    
    -- Atributos de Perfil
    Age,
    Gender,
    MaritalStatus,
    Education,
    EducationField,
    
    -- Outros atributos do funcionário
    DistanceFromHome,
    TotalWorkingYears,
    YearsAtCompany,
    YearsInCurrentRole,
    YearsSinceLastPromotion,
    YearsWithCurrManager
FROM raw_attrition;

-- Consulta rápida para checar a criação
SELECT * FROM DIM_EMPLOYEE LIMIT 5;

### 4.2. CRIAÇÃO DA DIMENSÃO DA INFORMAÇÃO DO CARGO (DIM_JOB_INFO)

Criação de Chave Substituta (SK) e seleção dos atributos de cargo e finanças, esta dimensão foca nas características do trabalho e remuneração.

In [0]:
%sql
-- Cria a tabela DIM_JOB_INFO no Catálogo Databricks
CREATE OR REPLACE TABLE DIM_JOB_INFO USING DELTA AS
SELECT
    -- Gerando a Chave Substituta (SK)
    row_number() OVER (ORDER BY EmployeeNumber) AS Job_SK,
    
    -- Chave de Negócio (para junção posterior)
    EmployeeNumber,
    
    -- Atributos de Cargo/Departamento
    Department,
    JobRole,
    JobLevel,
    JobInvolvement,
    JobSatisfaction,
    
    -- Atributos Financeiros
    DailyRate,
    HourlyRate,
    MonthlyRate,
    PercentSalaryHike
FROM raw_attrition;

-- Consulta rápida para checar a criação
SELECT * FROM DIM_JOB_INFO LIMIT 5;

### 4.3. CRIAÇÃO DA TABELA FATO (FACT_HR_MOVEMENT)

Esta é a tabela central. Ela armazena a métrica principal (Attrition_Flag) e as Chaves Estrangeiras (FKs) que se ligarão às dimensões.

É a Conversão da métrica Attrition ('Yes'/'No') para 0/1, e união das SKs.

In [0]:
%sql
-- Cria a tabela FACT_HR_MOVEMENT no Catálogo Databricks
CREATE OR REPLACE TABLE FACT_HR_MOVEMENT USING DELTA AS
SELECT
    -- Chave de Negócio do evento (o próprio EmployeeNumber)
    T1.EmployeeNumber,
    
    -- Chaves Estrangeiras (FKs)
    T2.Employee_SK,
    T3.Job_SK,
    
    -- Métrica: Transformação e pré-processamento (0/1) - Critério de ETL
    CASE
        WHEN T1.Attrition = 'Yes' THEN 1
        ELSE 0
    END AS Attrition_Flag,
    
    -- Outras métricas de interesse que você possa querer somar/medir
    T1.StandardHours,
    T1.MonthlyIncome,
    T1.PerformanceRating
FROM raw_attrition T1
-- Junção com as Dimensões (criadas nos passos anteriores) para obter as SKs
INNER JOIN DIM_EMPLOYEE T2 ON T1.EmployeeNumber = T2.EmployeeNumber
INNER JOIN DIM_JOB_INFO T3 ON T1.EmployeeNumber = T3.EmployeeNumber;

-- Consulta rápida para checar a criação
SELECT * FROM FACT_HR_MOVEMENT LIMIT 5;

## 5. Análise de Qualidade de Dados

Esta é a fase de Medir (do ciclo MVP) e exige a aplicação dos conceitos de Qualidade de Dados e Domínio/Integridade (DMBOK) . Vamos testar as regras que definimos no Catálogo de Dados.

Vamos focar em duas dimensões críticas: Completude (Nulos) e Validade (Domínio/Semântica).

### 5a.1. Teste de Completude (Verificação de Valores Nulos)

A completude é crucial, pois um valor nulo pode distorcer métricas de média e contagem.

Ação: Usar SQL para contar a proporção de nulos nas colunas mais relevantes (geralmente aquelas que definem perfis ou métricas).

In [0]:
%sql
-- Teste 1: Contagem de valores nulos nas colunas principais da DIM_EMPLOYEE
SELECT 
    'Age' AS Coluna, SUM(CASE WHEN Age IS NULL THEN 1 ELSE 0 END) AS Nulos, COUNT(*) AS Total, 
    (SUM(CASE WHEN Age IS NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS Percentual_Nulo
FROM DIM_EMPLOYEE
UNION ALL
SELECT 
    'TotalWorkingYears', SUM(CASE WHEN TotalWorkingYears IS NULL THEN 1 ELSE 0 END), COUNT(*), 
    (SUM(CASE WHEN TotalWorkingYears IS NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100
FROM DIM_EMPLOYEE
UNION ALL
-- Teste 2: Contagem de nulos nas métricas da FACT_HR_MOVEMENT
SELECT 
    'MonthlyIncome', SUM(CASE WHEN MonthlyIncome IS NULL THEN 1 ELSE 0 END), COUNT(*), 
    (SUM(CASE WHEN MonthlyIncome IS NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100
FROM FACT_HR_MOVEMENT;

-- Se todas as porcentagens forem 0, você pode documentar que o conjunto de dados é altamente 'curado' (tratado).

### 5a.2. Teste de Validade Semântica (Verificação de Domínio)

A validade verifica se os dados estão na faixa esperada, conforme a regra de negócio do RH.

Ação: Usar SQL para identificar registros que violam as regras de domínio (ex. Idade irreal ou métricas financeiras negativas).

In [0]:
%sql
-- Teste 3: Idade fora do domínio aceitável (regra de negócio: 18 <= Idade <= 60)
SELECT 
    'Age_Out_of_Range' AS Violacao, COUNT(*) AS Qtd_Violacoes
FROM DIM_EMPLOYEE
WHERE Age < 18 OR Age > 60
UNION ALL
-- Teste 4: Checando Consistência Categórica (Domínio) - A coluna Attrition_Flag SÓ DEVE ter 0 ou 1
SELECT 
    'Attrition_Flag_Invalida', COUNT(*)
FROM FACT_HR_MOVEMENT
WHERE Attrition_Flag NOT IN (0, 1)
UNION ALL
-- Teste 5: Valores Negativos em Métricas Financeiras (Regra Semântica)
SELECT 
    'DailyRate_Negativo', COUNT(*)
FROM DIM_JOB_INFO
WHERE DailyRate <= 0;

### 5a.3. Documentação da Qualidade

#### Métrica de Completude (Nulos):
* **Resultado:** Retornou 0 em todas as colunas críticas (Age, TotalWorkingYears, MonthlyIncome).
* **Conclusão:** O conjunto de dados apresenta 100% de **Completude**. Isso garante que as análises estatísticas não serão afetadas por viés causado por *missing values*.

#### Testes de Validade Semântica (Domínio/Integridade):
* **Resultado:** Retornou 0 em todos os testes (Age_Out_of_Range, Attrition_Flag_Invalida, DailyRate_Negativo).
* **Conclusão de Integridade:** A ausência de violações confirma a **Validade Semântica** dos dados em relação às regras de negócio. O sucesso do teste da coluna `Attrition_Flag` valida a **fase de Transformação (T)** do pipeline ETL, onde o dado categórico foi corretamente mapeado para a métrica binária (0/1).

**Resumo:** O conjunto de dados é altamente íntegro e 'curado', demonstrando alta **Qualidade de Dados** e **Conformidade de Domínio**. Nenhuma ação corretiva é necessária antes da fase de análise.

### 5b - Solução do Problema de Negócio

Agora, o foco é aplicar o poder do Esquema Estrela e do Spark-SQL para resolver o problema de negócio original: Analisar a Rotatividade (Attrition).

Vamos estruturar a análise em torno das três perguntas de negócio mais importantes para o RH, utilizando consultas que demonstrem:

1 - Uso de JOINs (para ligar Fato e Dimensões).

2 - Uso de GROUP BY e Funções Agregadas.

3 - Obtenção de insights de negócio.

### 5b.1. Análise Principal: Taxa de Rotatividade por Departamento

Esta análise usa FACT_HR_MOVEMENT e DIM_JOB_INFO e é a métrica mais crucial.

Pergunta de Negócio: Qual departamento possui a maior taxa de rotatividade (Attrition Rate)?

In [0]:
%sql
-- Analisando Rotatividade por Departamento (JOIN, SUM, COUNT, GROUP BY)
SELECT 
    CASE 
        WHEN T2.Department = 'Sales' THEN 'Vendas'
        WHEN T2.Department = 'Human Resources' THEN 'Recursos Humanos'
        WHEN T2.Department = 'Research & Development' THEN 'Pesquisa e Desenvolvimento'
        ELSE T2.Department
    END AS Departamento,
    SUM(T1.Attrition_Flag) AS Total_Saidas,
    COUNT(T1.EmployeeNumber) AS Total_Funcionarios,
    
    -- NOVA COLUNA: Apenas o valor numérico (double)
    ROUND((SUM(T1.Attrition_Flag) / COUNT(T1.EmployeeNumber)) * 100, 2) AS Taxa_Rotatividade_Valor
FROM 
    FACT_HR_MOVEMENT T1
INNER JOIN 
    DIM_JOB_INFO T2 
ON 
    T1.Job_SK = T2.Job_SK
GROUP BY 
    Departamento
ORDER BY 
    Taxa_Rotatividade_Valor DESC;

Databricks visualization. Run in Databricks to view.

**Percentual de Taxa de Rotatividade por Departamento**
- Vendas - 20.63%
- Recursos Humanos e Departamento - 19.05%
- Pesquisa e Desenvolvimento - 13.84%

### Análise para o Negócio (RH)

* **Insight de Negócio:** O Departamento de Vendas (`Vendas`) apresenta a maior taxa de rotatividade, com **20.63%**. Este valor é significativamente mais alto que o de P&D (13.84%), indicando que o problema está concentrado e deve ser a **prioridade de intervenção** do RH.

#### Documentação para o MVP (Discussão):
> "A taxa de Rotatividade por Departamento revelou uma discrepância crítica no setor de Vendas. Esta **agregação OLAP**, facilitada pelo **Esquema Estrela**, demonstrou que o problema é **localizado e operacional**, exigindo uma intervenção imediata e específica no Departamento de Vendas para mitigar o problema."

### 5b.2. Análise Comportamental: Rotatividade por Satisfação e Envolvimento

Esta análise correlaciona a rotatividade com as dimensões de envolvimento/satisfação, buscando a causa raiz do problema.

Pergunta de Negócio: Qual é o nível de Satisfação no Trabalho (JobSatisfaction) e Envolvimento (JobInvolvement) predominante entre os funcionários que pediram demissão?

In [0]:
%sql
-- Analisando o perfil de satisfação dos que saíram (JOIN, WHERE, AVG)
SELECT
    T2.JobSatisfaction,
    T2.JobInvolvement,
    COUNT(T1.EmployeeNumber) AS Qtd_Saidas_Por_Nivel
FROM
    FACT_HR_MOVEMENT T1
INNER JOIN 
    DIM_JOB_INFO T2 
ON 
    T1.Job_SK = T2.Job_SK
WHERE
    -- Filtra apenas funcionários que saíram (Attrition_Flag = 1)
    T1.Attrition_Flag = 1
GROUP BY
    T2.JobSatisfaction,
    T2.JobInvolvement
ORDER BY
    Qtd_Saidas_Por_Nivel DESC
LIMIT 5;

Databricks visualization. Run in Databricks to view.

### Análise para o Negócio (RH)

* **Insight de Negócio (Revisado):** A maior concentração de saídas (41 funcionários) está no perfil de **Satisfação Nível 3 (Média)** e **Envolvimento Nível 1 (Baixo)**. Este é um indicador de que a empresa perde funcionários que já demonstravam baixo envolvimento, mas que ainda mantinham uma satisfação média.
* **Discussão:** A prioridade do RH deve ser **intervir nas causas da Baixa Satisfação no Trabalho** (Nível 1) e tentar elevar o Envolvimento (Nível 3) nos funcionários que ficam, focando em reter o perfil de *média* satisfação antes que ele caia para o risco de saída.

> **Documentação para o MVP (Discussão):** O Gráfico de Colunas Empilhadas demonstrou visualmente a distribuição do volume de saídas por perfil comportamental, comprovando que o problema não é aleatório, mas sim um fator combinado de **Satisfação e Envolvimento**.

### 5b.3. Análise Financeira: Impacto do Salário (Renda Mensal) na Rotatividade

Esta análise usa a métrica financeira MonthlyIncome da Fato e a agrupa pela bandeira de rotatividade.

Pergunta de Negócio: Qual é a renda mensal média dos funcionários que saem versus a dos que permanecem?

In [0]:
%sql
-- Calculando o salário médio (MonthlyIncome) para quem saiu e quem ficou (AVG)
SELECT 
    CASE WHEN T1.Attrition_Flag = 1 THEN 'Saiu (Risco)' ELSE 'Ficou (Retenção)' END AS Status_Funcionario,
    
    -- Ajuste: Usa ROUND para limitar a duas casas decimais, ideal para dinheiro.
    ROUND(AVG(T1.MonthlyIncome), 2) AS Renda_Mensal_Media
FROM 
    FACT_HR_MOVEMENT T1
GROUP BY 
    Status_Funcionario
ORDER BY
    Renda_Mensal_Media ASC

Databricks visualization. Run in Databricks to view.

**Análise para o Negócio (RH)**

* **Insight de Negócio:** A Renda Mensal Média dos que saíram (R$ 4.787) é significativamente menor do que a dos que permaneceram (R$ 6.832). Este *insight* **prescritivo** sugere que a política de remuneração nas faixas salariais mais baixas é um fator de retenção crítico. A área de RH deve reavaliar os planos de aumento para este grupo para mitigar o risco.

**Documentação para o MVP (Discussão):**
> "A análise financeira utilizando a função agregada **AVG()** no Spark-SQL confirma que a Renda Mensal é um fator decisivo. A discrepância salarial indica um risco de **Rotatividade Voluntária** concentrada nas faixas de menor remuneração, demandando uma ação imediata na política salarial."

### 5b.4. Métrica de Referência: Taxa de Rotatividade Geral (Contexto BI)

Esta análise calcula a taxa de rotatividade total da empresa. Este valor é crucial para estabelecer um **benchmark** (ponto de comparação) no Dashboard, permitindo que a área de RH avalie se os 20.63% de Vendas estão significativamente acima da média geral.

In [0]:
%sql
SELECT 
    COUNT(*) AS Total_Registros, 
    COUNT(CASE WHEN T1.Attrition_Flag = 1 THEN 1 END) AS Total_Saidas,
    -- Calcula a taxa geral de rotatividade da empresa
    ROUND((COUNT(CASE WHEN T1.Attrition_Flag = 1 THEN 1 END) / COUNT(*)) * 100, 2) AS Taxa_Total_Geral_Pct
FROM 
    FACT_HR_MOVEMENT T1;

Databricks visualization. Run in Databricks to view.

Esta análise calcula a taxa de rotatividade total da empresa. Este valor é crucial para estabelecer um **benchmark** (ponto de comparação) no Dashboard, permitindo que a área de RH avalie se os **20.63% de Vendas** estão significativamente acima da média geral de **16.12%**.

#### Documentação para o MVP (Discussão):
> "A inclusão da **Taxa Total Geral (16.12%)** no Dashboard de BI fornece o contexto necessário para que os *stakeholders* avaliem o nível de gravidade das taxas departamentais (como os 20.63% de Vendas). A taxa geral atua como a **linha de base** para todas as análises de risco."

## 6. Entrega e Autoavaliação

### 6.1. Estrutura de Entrega e Evidências

Este MVP foi estruturado como um pipeline de dados em Nuvem (Databricks), usando o Esquema Estrela para análise. O trabalho demonstrou o ciclo de vida completo: ingestão, modelagem, ETL, análise de qualidade e geração de insights, atendendo ao requisito de **Qualidade Geral** do MVP.

#### Componentes e Evidências (Comprovação da Implementação)

* **Repositório e Código:** Todo o código-fonte (Notebook Databricks) está disponível em um Repositório Público no GitHub. O Notebook demonstra o uso de **PySpark/SQL** e o formato **Delta Lake**, comprovando a implementação do pipeline de dados em Nuvem.
* **Modelagem e Catálogo:** A desnormalização para o **Esquema Estrela** (FACT\_HR\_MOVEMENT + Dimensões) foi executada. O **Catálogo de Dados** (incluindo domínios e regras de qualidade) foi documentado na Etapa 3.
* **Comprovação da Carga e Qualidade:** Screenshots comprovam a execução bem-sucedida do pipeline de **ETL** e os resultados dos **Testes de Qualidade** (retorno zero violações para Completude e Validade Semântica).

### Resultados da Análise de Negócio (Etapa 5b)

| Análise | Métrica Principal | Insights Chave |
| :--- | :--- | :--- |
| Rotatividade por Departamento | **Vendas: 20.63%** Attrition Rate | O departamento de Vendas é o ponto de maior risco, **cerca de 7 pontos percentuais** acima de P&D (13.84%). |
| Perfil Comportamental | **41 saídas: Média Satisfação (3) + Baixo Envolvimento (1)** | A empresa perde talentos que já demonstravam baixo envolvimento, indicando que a insatisfação é o fator determinante para a saída. |
| Impacto da Renda | Renda Média (Saiu): R$ **4.787,09** vs. Renda Média (Ficou): R$ **6.832,74** | A Renda Mensal é um fator de retenção significativo; o grupo de menor salário tem maior propensão a sair. |

### 6.2. Autoavaliação e Reflexão

#### Atingimento dos Objetivos
O objetivo de estruturar e analisar a Rotatividade foi plenamente alcançado. O MVP foi capaz de responder de forma eficiente às perguntas de negócio, o que valida a modelagem como um **Esquema Estrela** dentro do ecossistema Databricks, provando sua adequação para consultas OLAP.

#### Dificuldades Encontradas e Aprendizado
A principal dificuldade inicial residiu na migração da arquitetura para a nuvem, especificamente a restrição de acesso ao diretório `/FileStore` do DBFS. Esse desafio técnico aprofundou o aprendizado sobre:
1.  **Arquitetura Data Lake:** A diferença entre Dados em Arquivo Bruto (CSV) e **Tabela Persistida em Catálogo** (Delta Lake), como exigido pela arquitetura moderna.
2.  **Governança:** A necessidade de usar o **Catálogo de Dados** como ponto de acesso, garantindo a governança do dado.

#### Trabalhos Futuros (Próximos MVPs)
Para aprimorar o portfólio de Data Science, os próximos passos seriam:
1.  **Análise Preditiva (MLOps):** O próximo MVP será aplicar modelos de Machine Learning (como Regressão Logística ou Random Forest) no Databricks para **prever quais funcionários têm alto risco de rotatividade**, usando as dimensões criadas (DIM\_EMPLOYEE, DIM\_JOB\_INFO) como *features*.
2.  **Operacionalização (DataOps):** Automatizar o pipeline de ETL/Análise em um **Job Recorrente** no Databricks, transformando o projeto pontual em um **Processo** contínuo de monitoramento de RH, alinhado aos conceitos de **DataOps** e **Gestão de Processos**.

## 7. Relatório Executivo e Recomendações Finais

###Visão Geral do Problema (Diagnóstico)

A análise no ambiente Databricks (Esquema Estrela) confirmou que a rotatividade é um **risco localizado e financeiro**.

| Setor de Risco | Métrica Principal | Insight Comportamental e Financeiro |
| :--- | :--- | :--- |
| **Vendas (Sales)** | **20.63%** (Maior risco) | A Rotatividade é alta em um grupo com **Média Satisfação (3)** e **Baixo Envolvimento (1)**, indicando problemas operacionais e de gestão de clima. |
| **Renda Média** | R$ **4.787,09** (para quem saiu) | Há um fator de retenção crítico ligado à faixa salarial mais baixa. |

### Recomendações (Ações Prescritivas)

Com base nas análises, as ações prioritárias para o RH são:
1.  **Ação Imediata (Financeira):** Reavaliar a política de remuneração para faixas salariais abaixo de R$ 5.000, pois é o grupo de maior propensão à saída voluntária.
2.  **Ação Operacional (Gestão):** Conduzir uma investigação de clima e gestão de desempenho especificamente no Departamento de **Vendas**, que apresenta taxas insustentáveis.